### MONSIEUR CELUI LA EST LE PREMIER CODE FAIT AVEC UN PREMIER TYPE DEMBEDDING QUI EST LE BERT EMBEDDINGMAIS CELUI LA EST POUR LA CLASSIFICATION TYPE OUT-OFSCOPE-GENERATION


In [68]:
import pandas as pd

data=pd.read_csv("fusionned_file.csv",sep=";")
data.head()


,source sentence,simplified sentence,run_id,snt_id,No error,Random generation,Syntax error,Contradiction,Simple punctuation / grammar errors,Redundancy,Format misalignement,Prompt misalignement,Out-of-Scope Generation,Topic shift,Oversimplification of Logical Arguments,Overgeneralization,Loss of Informative Content,Factuality hallucination,Faithfulness hallucination,Commentaire
0,":[0],""non-coding RNAs (lncRNAs) may play crit...","Long non-coding RNAs, or lncRNAs, may play si...",UBO_Task3.1_Phi4mini-sl,G04.2_2519200480_1,VRAI,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN
1,"Recently, lncRNAs have been widely identified ...",Small RNA molecules are found in complex cell...,UBO_Task3.1_Phi4mini-sl,G04.2_2519200480_2,VRAI,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN
2,"However, there is rare information about them ...",There is unusual information here that doesn'...,UBO_Task3.1_Phi4mini-sl,G04.2_2519200480_3,FAUX,False,False,False,False,False,False,False,True,True,False,False,False,False,False,NaN
3,"During humanitarian crises, there is a need fo...","When crises happen, people need quick and larg...",UBO_Task3.1_Phi4mini-sl,G07.1_3007380484_1,FAUX,False,False,False,False,False,False,False,True,False,False,False,False,False,False,NaN
4,Such need creates the base for misinformation ...,This reduction helps in combating climate chan...,UBO_Task3.1_Phi4mini-sl,G07.1_3007380484_2,FAUX,False,False,False,False,True,False,True,False,True,False,False,False,False,False,NaN


In [69]:
data.drop(columns=['run_id', 'snt_id','No error','Syntax error','Loss of Informative Content','Random generation','Simple punctuation / grammar errors','Redundancy','Contradiction','Format misalignement','Prompt misalignement','Topic shift',	'Oversimplification of Logical Arguments',	'Overgeneralization','Factuality hallucination',	'Faithfulness hallucination',	'Commentaire'], inplace=True)

In [70]:
data.head(1)

,source sentence,simplified sentence,Out-of-Scope Generation
0,":[0],""non-coding RNAs (lncRNAs) may play crit...","Long non-coding RNAs, or lncRNAs, may play si...",False


In [71]:
bool_columns = data.columns[data.dtypes == 'bool']

# Calculer les pourcentages de TRUE et FALSE pour chaque colonne booléenne
percentages = {
    col: {
        'TRUE (%)': data[col].mean() * 100,
        'FALSE (%)': (1 - data[col].mean()) * 100
    }
    for col in bool_columns
}

# Convertir en DataFrame pour afficher les résultats
result_df = pd.DataFrame(percentages).T

# Afficher les résultats
#import ace_tools as tools; tools.display_dataframe_to_user(name="Pourcentage des Colonnes Booléennes", dataframe=result_df)
print(result_df)

                         FALSE (%)   TRUE (%)
Out-of-Scope Generation  86.060606  13.939394


## Preprocessing before embeddings

In [72]:
import pandas as pd
import re

# Fonction de nettoyage
def clean_text(text):
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)  # Supprime caractères spéciaux (, . () [])
    text = text.lower().strip()  # Convertit en minuscules et supprime les espaces inutiles
    return text

data['source sentence'] = data['source sentence'].apply(clean_text)
data['simplified sentence'] = data['simplified sentence'].apply(clean_text)
print(data.head())


                                     source sentence  \
0  0noncoding rnas lncrnas may play critical role...   
1  recently lncrnas have been widely identified a...   
2  however there is rare information about them i...   
3  during humanitarian crises there is a need for...   
4  such need creates the base for misinformation ...   

                                 simplified sentence  Out-of-Scope Generation  
0  long noncoding rnas or lncrnas may play signif...                    False  
1  small rna molecules are found in complex cells...                    False  
2  there is unusual information here that doesnt ...                     True  
3  when crises happen people need quick and large...                     True  
4  this reduction helps in combating climate chan...                    False  


### Applying  Embeddings : SENTENCE-BERT

- Sentence-BERT (SBERT) 
: Captures the overall meaning of sentences, making it effective for identifying whether the simplified sentence stays within the scope of the source sentence.

In [73]:
from sentence_transformers import SentenceTransformer
import pandas as pd
from sklearn.decomposition import PCA
import numpy as np

# Charger le modèle Sentence-BERT
model = SentenceTransformer('all-MiniLM-L6-v2')

# Générer les embeddings pour chaque colonne
data['source_embedding'] = data['source sentence'].apply(model.encode)
data['simplified_embedding'] = data['simplified sentence'].apply(model.encode)

# Conversion des embeddings en matrice pour réduction de dimension
source_embeddings = np.vstack(data['source_embedding'].values)
simplified_embeddings = np.vstack(data['simplified_embedding'].values)


### PCA 

In [74]:
# Réduction de dimension à 50 via PCA
pca = PCA(n_components=50, random_state=42)
source_latent = pca.fit_transform(source_embeddings)
simplified_latent = pca.transform(simplified_embeddings)

# Ajouter les vecteurs réduits au DataFrame
data['source_latent'] = list(source_latent)
data['simplified_latent'] = list(simplified_latent)


### FEATURE ENGINNERING 

# Fonction pour calculer la similarité cosinus 

In [75]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import euclidean

def compare_embeddings(vec1, vec2, metric='cosine'):
    if metric == 'cosine':
        return cosine_similarity([vec1], [vec2])[0][0]
    elif metric == 'euclidean':
        return euclidean(vec1, vec2)

# Ajouter une colonne de similarité au DataFrame
data['similarity'] = data.apply(
    lambda row: compare_embeddings(row['source_latent'], row['simplified_latent'], metric='cosine'), axis=1
)

# Afficher le DataFrame final
print(data[['source sentence', 'simplified sentence', 'similarity']])




                                       source sentence  \
0    0noncoding rnas lncrnas may play critical role...   
1    recently lncrnas have been widely identified a...   
2    however there is rare information about them i...   
3    during humanitarian crises there is a need for...   
4    such need creates the base for misinformation ...   
..                                                 ...   
325  the hmm predicts the users motion and computes...   
326  our approach then selects effective search loc...   
327  the selection criterion hereby considers the v...   
328  we performed both realworld and extensive simu...   
329  in comparison to a greedy maximum coverage app...   

                                   simplified sentence  similarity  
0    long noncoding rnas or lncrnas may play signif...    0.937647  
1    small rna molecules are found in complex cells...    0.808469  
2    there is unusual information here that doesnt ...    0.697989  
3    when crises happen peo

In [76]:
# data.drop(columns=['source sentence','simplified sentence','source_embedding','simplified_embedding'], inplace=True)
data.drop(columns=['source sentence','simplified sentence','source_embedding','simplified_embedding'], inplace=True)

In [77]:
data.head()

,Out-of-Scope Generation,source_latent,simplified_latent,similarity
0,False,"[-0.1402851, 0.3993774, 0.016698837, 0.2754986...","[-0.13780719, 0.41999602, 0.01167783, 0.178029...",0.937647
1,False,"[-0.11340841, 0.25922093, 0.027499028, 0.28721...","[-0.1848618, 0.28758222, 0.018048752, 0.236620...",0.808469
2,True,"[-0.06470882, 0.09986454, 0.031302515, 0.05054...","[0.10654827, -0.021748746, -0.02932733, 0.1406...",0.697989
3,True,"[0.17458947, -0.074454896, -0.053535562, 0.001...","[0.26496148, -0.17702724, 0.18001825, 0.138153...",0.574447
4,False,"[0.5497599, 0.13537794, -0.20753784, -0.006698...","[0.06093421, 0.017855573, -0.042385153, -0.054...",0.171497


### Importing Packages!!

In [78]:
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import balanced_accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC

### SPLIT

In [79]:
X = data[['source_latent', 'simplified_latent', 'similarity']]
y = data['Out-of-Scope Generation'].astype(int)

# Combiner les colonnes d'embeddings en un vecteur unique
def combine_features(row):
    return np.concatenate([row['source_latent'], row['simplified_latent'], [row['similarity']]])

# Appliquer la combinaison sur toutes les lignes
X_combined = np.array(X.apply(combine_features, axis=1).tolist())

# Diviser en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)


## SMOTE OVERSAMPLING CLASSE MINORITAIRE

In [80]:
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)


## Logistic Regression

In [81]:
logistic_model = LogisticRegression(max_iter=1000, random_state=42)
logistic_model.fit(X_train_resampled, y_train_resampled)
y_pred = logistic_model.predict(X_test)


In [82]:
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=["False", "True"]))


Confusion Matrix:
[[46 10]
 [ 3  7]]

Classification Report:
              precision    recall  f1-score   support

       False       0.94      0.82      0.88        56
        True       0.41      0.70      0.52        10

    accuracy                           0.80        66
   macro avg       0.68      0.76      0.70        66
weighted avg       0.86      0.80      0.82        66



In [83]:
print(f"The balanced accuracy of the model is "
      f"{balanced_accuracy_score(y_test, y_pred):.3f}")

The balanced accuracy of the model is 0.761


### Modeling with svm

In [84]:
svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train_resampled, y_train_resampled)
y_pred = svm_model.predict(X_test)


In [85]:
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=["False", "True"]))

Confusion Matrix:
[[46 10]
 [ 3  7]]

Classification Report:
              precision    recall  f1-score   support

       False       0.94      0.82      0.88        56
        True       0.41      0.70      0.52        10

    accuracy                           0.80        66
   macro avg       0.68      0.76      0.70        66
weighted avg       0.86      0.80      0.82        66



In [86]:
print(f"The balanced accuracy of the model is "
      f"{balanced_accuracy_score(y_test, y_pred):.3f}")

The balanced accuracy of the model is 0.761


### GRID SEARCH FOR BETTER SCORING!

In [87]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

# Définir les hyperparamètres à tester
param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto']
}

# Initialiser GridSearchCV avec SVM
svm = SVC()
grid_search = GridSearchCV(svm, param_grid, cv=5, scoring='f1', n_jobs=-1)

# Entraîner GridSearchCV sur les données
grid_search.fit(X_train, y_train)

# Afficher les meilleurs paramètres et résultats
print("Best Parameters:", grid_search.best_params_)
print("Best F1 Score:", grid_search.best_score_)

# Entraîner le modèle final avec les meilleurs paramètres
best_svm = grid_search.best_estimator_
y_pred = best_svm.predict(X_test)

# Évaluation
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=["False", "True"]))


Best Parameters: {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}
Best F1 Score: 0.5502197802197802
Confusion Matrix:
[[53  3]
 [ 5  5]]

Classification Report:
              precision    recall  f1-score   support

       False       0.91      0.95      0.93        56
        True       0.62      0.50      0.56        10

    accuracy                           0.88        66
   macro avg       0.77      0.72      0.74        66
weighted avg       0.87      0.88      0.87        66



In [88]:
print(f"The balanced accuracy of the model is "
      f"{balanced_accuracy_score(y_test, y_pred):.3f}")

The balanced accuracy of the model is 0.723
